In [1]:
import pandas as pd
import json
import time


In [2]:
from openai import OpenAI


client = OpenAI()
#from error import APIConnectionError

In [3]:
df=pd.read_csv('../data/clean_data/Final_QandA_split_conversations_with_keywords.csv')

In [4]:
df.head()

,Chunked_Content,number of sentences,number of words,Keywords
0,when you eat it alone?. Good idea to eat it on...,2,69,"['pass', 'meal', 'home', 'nausea', 'honey', 'w..."
1,So they ate clay?. Act Charcoal will do the jo...,5,34,"['pass', 'meal', 'job', 'cold', 'neutral', 'cl..."
2,So when you have enough fat you feel nausea if...,1,13,"['eat', 'nausea', 'fat', 'feel']"
3,"So guys, water=bad?. I have dry skin even if i...",2,14,"['twice', 'water', '=', 'drink', 'skin', 'guy'..."
4,@ubeats1 could you please advice any source/bo...,8,110,"['detoxification', 'meal', 'trouble', 'literat..."


In [5]:
df.reset_index(level=0, inplace=True) 
df.rename(columns={'index':'id'}, inplace=True)

In [9]:
df.to_csv('../data/clean_data/final_data_with_IDs.csv', index=False)

In [6]:
documents = df.to_dict(orient='records')
documents[0:2]

[{'id': 0,
  'Chunked_Content': "when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks",
  'number of sentences': 2,
  'number of words': 69,
  'Keywords': "['pass', 'meal', 'home', 'nausea', 'honey', 'week', 'hunger', 'second', 'previous', 'btw', '20h+', 'kid', 'good', 'typical', 'idea', 'enhance', 'zero', '4', 'today', 'eat', 'yolk', 'egg', 'yesterday', 'wuth', 'hour', 'end']"},
 {'id': 1,
  'Chunked_Content': 'So they ate clay?. Act Charcoal will do the job?. 4,5 h passed I feel neutral, zero hunger and body warmed itself to confort temperature.. Before meal I felt colder than normal.. Thx Ring',
  'number of sentences': 5,
  'number of words': 34,
  'Keywords': "['pass', 'meal', 'job

In [8]:
prompt_template = """
You emulate a user of our health adviser application.
Formulate 5 questions this user might ask based on chunked content of conversation.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Chunked_Content: {Chunked_Content}


Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [9]:
prompt = prompt_template.format(**documents[0])

In [10]:
prompt

'You emulate a user of our health adviser application.\nFormulate 5 questions this user might ask based on chunked content of conversation.\nMake the questions specific to that chunk.\nThe record should contain the answer to the questions, and the questions should\nbe complete and not too short. Use as fewer words as possible from the record. \n\nThe record:\n\nChunked_Content: when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren\'t at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks\n\n\nProvide the output in parsable JSON without using code blocks:\n\n{"questions": ["question1", "question2", ..., "question5"]}'

In [11]:
print(_)

You emulate a user of our health adviser application.
Formulate 5 questions this user might ask based on chunked content of conversation.
Make the questions specific to that chunk.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

Chunked_Content: when you eat it alone?. Good idea to eat it once wuth honey and once alone My kids eat it with the meal.They aren't at home btw meals I will do it during week end Yesterday I had nausea after several hours passed from second meal.Today I had zero hunger until 20h+ passed after previous that meal Today I made only one typical meal enhanced with 4 egg yolks


Provide the output in parsable JSON without using code blocks:

{"questions": ["question1", "question2", ..., "question5"]}


In [12]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [13]:
questions = llm(prompt)
questions

'{"questions": ["Is it a good idea to eat this food alone or with honey?", "How do your kids typically eat this food?", "Will you be trying this during the weekend?", "What symptoms did you experience after your meal yesterday?", "What did you do differently with your meal today?"]}'

In [14]:
import json

In [15]:
json.loads(questions)

{'questions': ['Is it a good idea to eat this food alone or with honey?',
  'How do your kids typically eat this food?',
  'Will you be trying this during the weekend?',
  'What symptoms did you experience after your meal yesterday?',
  'What did you do differently with your meal today?']}

In [61]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [62]:
from tqdm.auto import tqdm

In [63]:
results = {}


In [64]:
len(documents)

49006

In [20]:
# Generationg questions for 49k documents was taking too long

In [21]:
# get 3000 random documents out of 49006 documents for evaluation
import random
number = 3000                           # set the number to select here.
random_docs = random.sample(documents,number)
len(random_docs)


3000

## Parallelarization
it did not work

In [59]:
len(random_docs[1000:2000])

1000

In [71]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def process_document(doc):
    doc_id = doc['id']
    if doc_id in results:
        return None  # Skip this document if already processed
    try:
        questions_raw = generate_questions(doc)
        questions = json.loads(questions_raw)
        return (doc_id, questions['questions'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing doc_id {doc_id}: {e}")
        return None

# Track time
t0 = time.time()

# Create a ThreadPoolExecutor with a suitable number of workers
with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit all tasks
    
    futures = {executor.submit(process_document, doc): doc for doc in tqdm(random_docs[1000:3000])}

    total_tasks = len(futures)
    # Collect results as they complete
    for future in tqdm(as_completed(futures)):
        result = future.result()
        if result is not None:
            doc_id, questions = result
            results[doc_id] = questions

# End time
t1 = time.time()
# Print time taken
print(f"Time taken: {(t1 - t0)/60} minutes")


  0%|          | 0/1000 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Error processing doc_id 1540: Expecting value: line 7 column 3 (char 243)
Error processing doc_id 12526: Expecting value: line 5 column 3 (char 148)
Time taken: 5.546072518825531 minutes


In [72]:
len(results)


1993

In [25]:
import time
t0=time.time()
for doc in tqdm(random_docs[:1000]): 
    doc_id = doc['id']
    if doc_id in results:
        continue
    try:
        questions_raw = generate_questions(doc)
        questions = json.loads(questions_raw)  # If an error happens here, the next line will not run
        results[doc_id] = questions['questions']  # This line will be skipped if there's an error in json.loads()
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing doc_id {doc_id}: {e}")
    
t1=time.time()
print((t0-t1)/60)

  0%|          | 0/1000 [00:00<?, ?it/s]

-141.73016977707545


In [67]:
len(results)
type(results)

dict

In [73]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [74]:
final_results[0]

(13369, 'Does any kind of fermentation cause k2 to increase?')

In [75]:
df_results2 = pd.DataFrame(final_results, columns=['id', 'question'])

In [78]:
df_results2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9916 entries, 0 to 9915
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        9916 non-null   int64 
 1   question  9916 non-null   object
dtypes: int64(1), object(1)
memory usage: 155.1+ KB


In [80]:
df=pd.concat([df_results, df_results2], axis=0)
df.head()

,id,question
0,8436,What are the two lube formulas you mentioned?
1,8436,What is the maximum amount of fat I should con...
2,8436,How many days did it take you to reach a weigh...
3,8436,"Did you triple your intake of something, and w..."
4,8436,Can you tell me more about what else you inclu...


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14910 entries, 0 to 9915
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        14910 non-null  int64 
 1   question  14910 non-null  object
dtypes: int64(1), object(1)
memory usage: 349.5+ KB


In [77]:
#df_results = pd.DataFrame(final_results, columns=['id', 'question'])
df_results.head()

,id,question
0,8436,What are the two lube formulas you mentioned?
1,8436,What is the maximum amount of fat I should con...
2,8436,How many days did it take you to reach a weigh...
3,8436,"Did you triple your intake of something, and w..."
4,8436,Can you tell me more about what else you inclu...


In [83]:
df.to_csv('../data/clean_data/ground-truth-data.csv', index=False)

In [84]:
!head '../data/clean_data/ground-truth-data.csv'

id,question
8436,What are the two lube formulas you mentioned?
8436,What is the maximum amount of fat I should consume daily?
8436,How many days did it take you to reach a weight of 66 kg?
8436,"Did you triple your intake of something, and what was it?"
8436,Can you tell me more about what else you included in your diet?
23461,Are there additional resources available in AV books?
23461,Can you find more information in the AV books?
23461,Is there extensive reading material on this topic in AV books?
23461,What specific AV books contain more information about this?
